## Problem statement

The deliverable for Milestone 3 is a Jupyter Notebook (preferably hosted on GitHub) showing a workflow to set up FCN and U-Net models for training using NWPU-RESISC45 lake images and corresponding labels. This will mostly test your understanding of the generic workflow of setting up multiple models for sequential training, in order to evaluate and compare model outputs and ultimately decide which model is best for the task, as well as how to implement custom conditional random fields for refining labels and segmentations in Milestones 4 and 5.

### import(s)

In [ ]:
import zipfile
import requests
import rasterio
import matplotlib
import numpy as np
import json, os, glob
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Concatenate, Conv2D, Conv2DTranspose, MaxPooling2D

### utils

In [ ]:
def unzip(f):
    """
    f = file to be unzipped
    """    
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall()


def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

### prepare sentinel 2 data

In [ ]:
#imagery
file_id = '1iMfIjr_ul49Ghs2ewazjCt8HMPfhY47h'
destination = 's2cloudless_imagery.zip'
download_file_from_google_drive(file_id, destination)


#labels
file_id = '1c7MpwKVejoUuW9F2UaF_vps8Vq2RZRfR'
destination = 's2cloudless_label_imagery.zip'
download_file_from_google_drive(file_id, destination)

# unzip
unzip('s2cloudless_imagery.zip')
unzip('s2cloudless_label_imagery.zip')


# remove zip files
os.remove('s2cloudless_imagery.zip')
os.remove('s2cloudless_label_imagery.zip')

In [ ]:
BATCH_SIZE=16
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    fill_mode = "constant", cval=0.0)

img_generator = train_datagen.flow_from_directory(
        's2cloudless_imagery',
        target_size=(512, 512),
        batch_size=BATCH_SIZE,
        class_mode=None, seed=111, shuffle=False)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    fill_mode = "constant", cval=0.0)

mask_generator = test_datagen.flow_from_directory(
        's2cloudless_label_imagery',
        target_size=(512, 512),
        batch_size=BATCH_SIZE,
        class_mode=None, seed=111, shuffle=False)

train_generator = (pair for pair in zip(img_generator, mask_generator))

In [ ]:
img_batch, mask_batch = next(train_generator)

def get_pair(i):
    img = img_batch[i].astype('uint8')/255
    msk = np.max(mask_batch[i], axis=2)/255
    msk[msk>=.5]  = 1
    msk[msk<.5] = 0
    msk = np.stack((msk,)*3, axis=-1)
    return np.concatenate([img, msk], axis = 1)

In [ ]:
plt.figure(figsize=(12,6))
plt.axis('off')
print(f"{get_pair(0).shape}")
plt.imshow(get_pair(0))

plt.figure(figsize=(12,6))
plt.axis('off')
plt.imshow(get_pair(11))


### U-Net model

In [ ]:
inputs = Input((1216, 1920, 3))
_tensor = inputs
  
#down sampling 
f = 8 #initially, use an 8-pixel kernel for the convolutional filter
layers = []

#cycle through 6 iterations, each time reusing '_tensor' 
#on each iteration ...
#pass through 2 convolutional blocks, append to the 'layers' output list
#then apply max pooling, and double the filter size for the next iteration
for i in range(0, 6):
   _tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)
   _tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)
   layers.append(_tensor)
   _tensor = MaxPooling2D() (_tensor)
   f = f*2
   print(_tensor.shape)


#bottleneck 
ff2 = 64 ##use an 64-pixel kernel for the convolutional filter  
j = len(layers) - 1
_tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)
_tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)
_tensor = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (_tensor)
# use concatenate to merge feature maps
_tensor = Concatenate(axis=3)([_tensor, layers[j]])
j = j -1 
print(_tensor.shape)

#upsampling 
for i in range(0, 5):
    ff2 = ff2//2
    f = f // 2 
    _tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)
    _tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)
    _tensor = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (_tensor)
    _tensor = Concatenate(axis=3)([_tensor, layers[j]])
    print(f"j@decoder: {j}")
    print(_tensor.shape)
    j = j - 1

_tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)
_tensor = Conv2D(f, 3, activation='relu', padding='same') (_tensor)  
_tensor.shape
print(_tensor.shape)

outputs = Conv2D(3, 1, activation='sigmoid') (_tensor)
print(outputs.shape)

In [ ]:
model = Model(inputs=[inputs], outputs=[outputs])
model.summary()